### This notebook demonstrates how to evaluate accuracy

In [1]:
from sklearn.neighbors import NearestNeighbors
from TfidfVectorizer import TfidfVectorizer
from tqdm import tqdm
from utils import *

data = load_data_from_json('data/data_with_augmented.json')
data_test = load_data_from_json('data/test_200.json')

In [2]:
vectorizer = TfidfVectorizer(stop_words=None, stemmer=None, norm=False)

X_tfidf = vectorizer.fit_transform(data)
words = vectorizer.words

knn = NearestNeighbors(metric='cosine')
knn.fit(X_tfidf)

y_pred = []
y_gold = []
for word, defi in tqdm(data_test.items()):
    query = list(defi)[0]
    query = vectorizer.transform(query)

    prediction = search(query, knn, words, n=1000)

    y_pred.append(prediction)
    y_gold.append(word)

acc_1, acc_10, acc_100, median_rank, var_rank = evaluate(y_pred, y_gold)